# 自己的情感分析任务
针对自己的数据集的情感分析任务，加载之前训练好的预训练模型，用自己的游记文本数据再次训练模型，学习游记文本中的语义


# 预处理自己的游记文本数据集
对标注好的游记文本数据进行打乱处理，划分好训练集、验证集和测试集

In [10]:
import torch
import torch.utils.data as D
import pandas as pd
from datasets import load_dataset

In [11]:
dataset = load_dataset('csv',data_files='../data/MyDataset/dataset.csv')

Using custom data configuration default-f5f7c946b412aa35


Extracting data files: 100%|██████████| 1/1 [00:00<00:00, 1095.69it/s]
0 tables [00:00, ? tables/s]/home/chenli/anaconda3/envs/pytorch/lib/python3.9/site-packages/datasets/download/streaming_download_manager.py:697: FutureWarning: the 'mangle_dupe_cols' keyword is deprecated and will be removed in a future version. Please take steps to stop the use of 'mangle_dupe_cols'
  return pd.read_csv(xopen(filepath_or_buffer, "rb", use_auth_token=use_auth_token), **kwargs)


Dataset csv downloaded and prepared to /home/chenli/.cache/huggingface/datasets/csv/default-f5f7c946b412aa35/0.0.0/652c3096f041ee27b04d2232d41f10547a8fecda3e284a79a0ec4053c916ef7a. Subsequent calls will reuse this data.


100%|██████████| 1/1 [00:00<00:00, 585.55it/s]


In [12]:
dataset = dataset['train']

In [16]:
len(dataset)

2362

In [13]:
shuffle_dataset = dataset.shuffle(9) # 将数据打乱，数值越大，混乱程度越大

In [18]:
train_size = int(0.8 * len(shuffle_dataset))
valid_size = int(0.1 * len(shuffle_dataset))
test_size = len(shuffle_dataset) - train_size - valid_size
train_dataset,valid_dataset,test_dataset = torch.utils.data.random_split(shuffle_dataset, [train_size,valid_size,test_size])

In [19]:
print(train_size)
print(valid_size)
print(test_size)

1889
236
237


In [21]:
print(shuffle_dataset[test_dataset.indices])

{'text': ['2021年11月19日，清晨6点出发，前往安徽的景区。经过近6小时长途跋涉，下午1点半左右，到达塔川，停车、买票等等，约下午2点进入景区。塔川村依山而建，飞檐翘角的古民居，层层叠叠，错落有致，远远望去，就好像一座宝塔矗立在山谷之中，此为塔之来历。一条清澈的小溪穿村潺潺流过，给小村带来无穷灵气，此为川之来历。合起来，即为塔川。塔川最出名的要属秋色，被广大摄影爱好者推崇为中国三大秋色观赏景区之一。每到秋季，满山的树叶色彩斑斓，从黄、浅红到火红，极富层次感，是安徽省内秋色绝佳的地方。皖南的秋韵，每一缕秋红都写满了诗情，每一片橘黄都沾染了画意。塔川以塔川秋色最为出名，秋天的菊花、深秋的红叶都值得一看，红叶主要是乌桕树，十月中下旬塔川开始入秋，绿色、黄色、红色的叶子开始在漫山遍野的乌桕树上交织出斑斓的色彩，在村庄里升起的袅袅炊烟中，像极了一幅泼墨风景画。塔川以秋色著称，有着中国三大秋色之一的美誉。每到秋季，经过霜降，塔川数万株乌桕树开始呈现迷人色彩，绿色，黄色，橘红色，粉色，大红色，层林尽染，就像上帝打翻了调色板般，色彩斑斓，远看就像一副鲜艳的油画，塔川也因此名扬。2021年11月20日，今天游玩目的地是和。早上先去了木梨硔，木梨硔村是百佳摄影点之一，该村生态绝佳，地形独特，地处山脊，三面悬空，徽风浓郁，被誉为最美的高山村落。民间有传说，黄山的高山深处有一个古村落——木梨硔，是黄山四绝之外的第五绝。要进村子，还得爬八百台阶，步行近半个小时。那里只是现代旅游圣地黄山界内的一个古村落，路很不好走，车是开不到村子的，人们喝的是大自然赋予的泉水，吃的只有当地农家饭农家菜。想去木梨硔，一般只有自驾租车或包车，村口有个收费站收停车费的景区是免费的。从山下的收费口起，山路很烂，非铺装路，一路那个颠，这还不算，路还很窄，两车交会太难了，建议新手就不要去受罪了！木梨硔主要是看日落日出及云雾，所以，最好是能住宿在山上，可是呢也很纠结的，就人走上去已经很累了，还要提着箱子什么的，受罪啊！当然可以让村民帮我挑上去，花钱呗，嘻嘻景色是没说的了，不虚此行！朝朱熹赞誉呈坎呈坎双贤里，江南第一村。呈坎现今还完整保存着1800年前东汉时期建村的八卦风水布局和建筑街巷、村落风水文化、三街九十九巷的古村落以及东汉、唐、宋、元、明、清等历代保存完好古建筑、遗址180多处和部分出土的新石器、青铜

In [22]:
print(shuffle_dataset[valid_dataset.indices])

{'text': ['黄山的屯溪香茗酒店，五星级三百八，早餐质量不错。六点开始，还是没有人。就是毕竟是国产品牌的酒店，经营和品牌策划有点混乱，细节还是，没什么出彩的地方。其实酒店的面积不小，但是庭院园林和绿化，相当糟糕，布局杂乱。嗯，尤其是园林布局完全没有章法不产生美感。推广价一块钱，请全国人民有景点，这是西递，原价104块，钱现在只要一块钱。2010年来过西堤，之后就再也没来过，这里面现在，真系已经基本没有了，原版的，木雕砖雕石雕，估计都被收走了，剩下一大堆复刻版和盗版的。庭院格局老房子，倒是依旧，但是老房子的生活方式也改变了，大部分人也并不从事农耕，都是旅游业，里面都是假古董。2010年来的时候还能请一个不错的导游，现在已经听不到了，导游都是程序化的套话。歙县火车站，汽车站旁边的小巷子里，还能依稀寻找到一些，皖南风格的煎饼，有梅干菜馅儿的笋子馅儿的，还有肥肉馅儿的，还有芹菜馅儿的，嗯，看到是外地人就乱报价格，要两块五一张了。正好赶上新安江，三潭地区的枇杷节。家家户户到处都是枇杷，价格也很混乱，从歙县县城礼包价，567块钱，到，嗯，三潭地区报价两块钱三块钱，十块钱五块钱都有。还是以前骑车来，这个湘江边的樟树，还好这个景点不收钱。因为昨晚暴雨所以的水有点浑浊。先将三潭地区的枇杷还是挺有名的，但是因为枇杷这种水果，不好保存，也就这么几天的新鲜期，所以今年丰产也卖不上价格。越往后面走琵琶越多，遍地满山遍野家家户户，都是屁吧！走到农民家的，二楼阁楼想拍张照片，结果被农民伯伯追到说要收费，除非买他家的枇杷。其实夏天的新疆也还是挺美的，就是下午光线不好，要是清晨一定很漂亮。二深度结束新安江画廊，还好，没收费，春天油菜花开的时候这条路是要收钱的。', '沿着金鞭溪大道走进中国第一个国家森林公园群峰拔地而起如巨笋傲指苍穹森林公园位于西北部，属武陵山脉，以峰称奇、以水显幽、以林见秀的山水仙境森林公园大门。景观大道两边排满中国领导人及各界著名人士的书法石碑拍摄了部分，请先观赏一下吧。情缘推向世界，国画大师吴冠中功不可没。1979年10月，他应邀来为人民大会堂厅绘制巨型风景画稿，11月，他专程到采风，有缘来到林场。他被这里的风光深深吸引，写下了游记散文养在深闺人未识，发表在1980年元旦的日报上。他把当成一颗失落在的风景明珠，他在文中这样写道为了探求绘画之美，我辛辛苦苦踏过不少名山，觉

In [23]:
pd.DataFrame(shuffle_dataset[train_dataset.indices]).to_csv('../data/MyDataset/train_dataset.csv',index=False)

In [24]:
pd.DataFrame(shuffle_dataset[valid_dataset.indices]).to_csv('../data/MyDataset/valid_dataset.csv',index=False)

In [25]:
pd.DataFrame(shuffle_dataset[test_dataset.indices]).to_csv('../data/MyDataset/test_dataset.csv',index=False)

# 具体的情感分类任务
针对自己数据集的情感分类任务

## DataSet类加载数据
Pytorch 通过 Dataset 类和 DataLoader 类处理数据集和加载数据构建 batch。因此我们首先需要编写继承自 Dataset 类的自定义数据集用于组织样本和标签

In [1]:
import torch
from datasets import load_dataset
import warnings
warnings.filterwarnings("ignore")

#定义数据集,方便后续模型读取批量数据。
class Dataset(torch.utils.data.Dataset):
    def __init__(self, data_type):
        self.data = self.load_data(data_type)
    
    # 核心要变的就是load_data这部分函数
    # 改造成适合自己任务的数据集
    def load_data(self, data_type):
        # 先加载本地数据集
        tmp_dataset = load_dataset('csv',data_files=data_type,split='train')
        Data = {}
        for idx, line in enumerate(tmp_dataset):
            sample = line
            Data[idx] = sample
        return Data

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return self.data[idx]    

/home/chenli/anaconda3/envs/pytorch/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
train_data = Dataset('../data/MyDataset/train_dataset.csv')
valid_data = Dataset('../data/MyDataset/valid_dataset.csv')
test_data = Dataset('../data/MyDataset/test_dataset.csv')

Using custom data configuration default-f4542a74482b2188
Found cached dataset csv (/home/chenli/.cache/huggingface/datasets/csv/default-f4542a74482b2188/0.0.0/652c3096f041ee27b04d2232d41f10547a8fecda3e284a79a0ec4053c916ef7a)
Using custom data configuration default-9358b381c3caed87
Found cached dataset csv (/home/chenli/.cache/huggingface/datasets/csv/default-9358b381c3caed87/0.0.0/652c3096f041ee27b04d2232d41f10547a8fecda3e284a79a0ec4053c916ef7a)
Using custom data configuration default-4bbbcf0261746e3e
Found cached dataset csv (/home/chenli/.cache/huggingface/datasets/csv/default-4bbbcf0261746e3e/0.0.0/652c3096f041ee27b04d2232d41f10547a8fecda3e284a79a0ec4053c916ef7a)


In [3]:
print(len(train_data))
print(len(valid_data))
print(len(test_data))

1889
236
237


In [4]:
print(train_data[0])
print(valid_data[0])
print(test_data[0])

{'text': '现在旅行成为朋友圈必晒元素之一，有人晒单人的自由自在，有人晒情侣的甜甜蜜蜜，有人晒亲子的幸福美满，却很少人晒带着爸妈去旅行。相信很多人都碰到过，说带爸妈出去玩，他们会说自己老了，不经折腾或者是不喜欢外面的热闹。其实很多时候他们是为了帮我们省钱省心。在国外旅行最常看见的就是老年夫妇结伴周游世界，为什么我们的爸妈不可以？这一次我就给爸妈安排了三天两晚。提起科技馆，大多数人可能会想，那不是带孩子去普及教育的地方吗？我想说，是时候打破传统观念啦。你带孩子去科技馆是涨见识，但在，带着父母那除了涨见识，还有返老还童，放松身心，重视身体的额外功效哦。厦门科技馆一共有5大展区，海洋馆、探索馆、和谐馆、创造馆，以及儿童馆，各具特色！共有400项体验项目。作为体验过的人，针对父母的情况，我选择从静到动的方式带他们这样玩。首先打卡的是飞越影院即4D动感影院，在这个鲜活的环境剧场上演了惊险、逼真的史前历险，犹如身临其境，亲眼见证远古时代小恐龙和妈妈的故事。4D影院对于年轻人来说不再新鲜，但是对于父母一代人来说，他们很少接触。坐着就能感受迎面吹来真实的风和雨，以第一视角随着座椅沉浸式体验，安全中带着小刺激，正好为我们的探索之旅热热身。接着我带他们去了和谐发展馆，这可是探索生命科学的地方，发现有关人身体的各个秘密。作为孩子，是不是一直苦于没法叫动父母去医院做体检呢？哈哈，让他们在体验过程中主动了解人体器官的功能，通过好玩的设施都能轻松get自己身体健康的知识点哦。比如上面的视力挑战不是医院枯燥的视力检查，却在电子射击中，考验了爸妈的眼力和手脚反应能力，同时也给我们做孩子的提个醒，该注意父母身体健康问题。再比如这个张大的嘴，是叫食物的旅行体验项目。我们在其中寻找蛀牙和味蕾，可以体验这个巨人胃的蠕动，真实地感受到胃部工作的情况。父母平时饮食有时不太讲究，这回身临其境上了一堂难忘的课。魔幻大滚轮出场啦。只需站在大滚轮内，由慢至快行走或奔跑，就可观察自己的心律血压的实时变化，时刻监控自己的身体状况，避免高负荷的运动对健康造成损伤。真的，平时和爸妈说千万遍，都不如这样玩一下让他们了解深刻。如果说和谐发展馆是热身，让爸妈产生兴趣去尝试，那么探索发现馆无疑让他们挑战值升高。人气爆棚的磁电大舞台可是国内第一个以舞台剧表演形式存在的哦。没想到现场互动的时候，我妈第一个举手参与。居然还被主持人

## DataLoader
创建好数据集之后，就需要通过 DataLoader 库来按批 (batch) 加载数据，将样本组织成模型可以接受的输入格式。对于 NLP 任务来说，这个环节就是对一个 batch 中的句子（这里是“句子对”）按照预训练模型的要求进行编码（包括 Padding、截断等操作）通过在 DataLoader 中设置批处理函数 collate_fn 来实现。

In [4]:
import torch
from torch.utils.data import DataLoader
from transformers import AutoTokenizer

checkpoint = "schen/longformer-chinese-base-4096"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

# 批量处理函数
def collote_fn(batch_samples):
    batch_text= []
    batch_label = []
    for sample in batch_samples:
        batch_text.append(sample['text'])
        batch_label.append(int(sample['label']))
    X = tokenizer(
        batch_text, 
        padding=True, 
        truncation=True, 
        return_tensors="pt"
    )
    y = torch.tensor(batch_label)
    return X, y

In [5]:
# batch_size=2跑完了，batch_size=4 跑不动
# 改为1条1条跑
train_dataloader = DataLoader(train_data, batch_size=1, shuffle=True, collate_fn=collote_fn)
valid_dataloader = DataLoader(valid_data, batch_size=1, shuffle=True, collate_fn=collote_fn)
test_dataloader = DataLoader(test_data, batch_size=1, shuffle=True, collate_fn=collote_fn)

batch_X, batch_y = next(iter(train_dataloader))
print('batch_X shape:', {k: v.shape for k, v in batch_X.items()})
print('batch_y shape:', batch_y.shape)
print(batch_X)
print(batch_y)

batch_X shape: {'input_ids': torch.Size([1, 2113]), 'token_type_ids': torch.Size([1, 2113]), 'attention_mask': torch.Size([1, 2113])}
batch_y shape: torch.Size([1])
{'input_ids': tensor([[  101,  4638,  5401,  ...,  8168, 11430,   102]]), 'token_type_ids': tensor([[0, 0, 0,  ..., 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1,  ..., 1, 1, 1]])}
tensor([1])


In [6]:
print(len(train_dataloader))
print(len(valid_dataloader))
print(len(test_dataloader))

1889
236
237


In [7]:
for batch, (X, y) in enumerate(train_dataloader, start=1):
    # print(batch)
    if(batch%200==199):
        print(batch +1 )

200
400
600
800
1000
1200
1400
1600
1800


## 模型加载（定义）
加载前要下定义模型 <br/>
这里不再是模型的定义了，模型已经进行了一轮预训练，可以直接加载进来。

In [8]:
from torch import nn
from transformers import AutoModel

device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f'Using {device} device')
checkpoint = "schen/longformer-chinese-base-4096"

class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.bert_encoder = AutoModel.from_pretrained(checkpoint)
        self.classifier = nn.Linear(768, 2)

    def forward(self, x):
        bert_output = self.bert_encoder(**x)
        cls_vectors = bert_output.last_hidden_state[:, 0]
        logits = self.classifier(cls_vectors)
        return logits

model = NeuralNetwork().to(device)
print(model)

Using cpu device


Some weights of the model checkpoint at schen/longformer-chinese-base-4096 were not used when initializing BertModel: ['bert.encoder.layer.9.attention.self.query_global.weight', 'bert.encoder.layer.5.attention.self.key_global.weight', 'bert.encoder.layer.11.attention.self.value_global.weight', 'cls.predictions.transform.dense.bias', 'bert.encoder.layer.9.attention.self.value_global.bias', 'bert.encoder.layer.0.attention.self.key_global.bias', 'bert.encoder.layer.2.attention.self.query_global.weight', 'bert.encoder.layer.8.attention.self.query_global.weight', 'bert.encoder.layer.7.attention.self.value_global.weight', 'bert.encoder.layer.7.attention.self.key_global.weight', 'bert.encoder.layer.3.attention.self.value_global.bias', 'bert.encoder.layer.0.attention.self.value_global.bias', 'bert.encoder.layer.7.attention.self.query_global.weight', 'bert.encoder.layer.2.attention.self.query_global.bias', 'bert.encoder.layer.3.attention.self.query_global.bias', 'bert.encoder.layer.2.attention.

NeuralNetwork(
  (bert_encoder): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(21128, 768, padding_idx=0)
      (position_embeddings): Embedding(4096, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_af

In [9]:
# 有模型就加载，没有的话就训练
model.load_state_dict(torch.load('/home/chenli/pre_model/2022.10.22/epoch_4_valid_acc_95.0_model_weights.bin'))
model.eval()

NeuralNetwork(
  (bert_encoder): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(21128, 768, padding_idx=0)
      (position_embeddings): Embedding(4096, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_af

## 模型训练
针对自己的数据集进行训练 <br/>
我们将每一轮 Epoch 分为训练循环和验证/测试循环。在训练循环中计算损失、优化模型的参数，在验证/测试循环中评估模型的性能

In [11]:
from tqdm.auto import tqdm # 显示它的进度条，会更好看点

# 参数解释
# dataloader ： 批量数据的loader
# model : 定义的模型
# loss_fn ： 定义的损失函数
# optimizer ：优化器
# lr_scheduler ： 学习率根据步数会下降，动态变化的。如果用一个固定的学习率，其实是没有这种随着迭代次数下降的效果好的
# epoch ：训练的轮次
# total_loss ：整体loss的情况
def train_loop(dataloader, model, loss_fn, optimizer, lr_scheduler, epoch, total_loss):
    progress_bar = tqdm(range(len(dataloader)))
    progress_bar.set_description(f'loss: {0:>7f}')
    finish_batch_num = (epoch-1)*len(dataloader)
    
    model.train()
    for batch, (X, y) in enumerate(dataloader, start=1):
        X, y = X.to(device), y.to(device)
        pred = model(X)
        loss = loss_fn(pred, y)

        optimizer.zero_grad() # 把之前的梯度都清掉
        loss.backward() # 向后传播
        optimizer.step() # 算完梯度下降之后更改参数
        lr_scheduler.step() # 对学习率进行调整

        total_loss += loss.item() # 统计一下整体的loss
        # batch=2时，每100输出一次loss
        if batch % 200 == 199:
            print('[%d,%5d] running_loss:%.3f' % (epoch,batch+1,total_loss/(finish_batch_num + batch)))
        progress_bar.set_description(f'loss: {total_loss/(finish_batch_num + batch):>7f}')
        progress_bar.update(1)
        
    return total_loss

def test_loop(dataloader, model, mode='Test'):
    assert mode in ['Valid', 'Test']
    size = len(dataloader.dataset)
    correct = 0

    model.eval()
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()

    correct /= size
    print(f"{mode} Accuracy: {(100*correct):>0.1f}%\n")
    return correct

# 2022.11.3 训练结果
用自己的数据集进行训练，加载模型。<br/>
batch_size=2，共945个批次，轮次=10

In [43]:
from transformers import AdamW, get_scheduler

learning_rate = 1e-5 # 定义学习率
epoch_num = 10 # 轮次定义

loss_fn = nn.CrossEntropyLoss() # 损失函数，交叉熵
optimizer = AdamW(model.parameters(), lr=learning_rate) # Adamw一个常用的优化器
lr_scheduler = get_scheduler(
    "linear",# 使用线性的方式，慢慢往下降
    optimizer=optimizer,
    num_warmup_steps=0,
    num_training_steps=epoch_num*len(train_dataloader),
)

total_loss = 0.
best_acc = 0.
for t in range(epoch_num):
    print(f"Epoch {t+1}/{epoch_num}\n-------------------------------")
    total_loss = train_loop(train_dataloader, model, loss_fn, optimizer, lr_scheduler, t+1, total_loss)
    valid_acc = test_loop(valid_dataloader, model, mode='Valid')
    if valid_acc > best_acc:
        best_acc = valid_acc
        print('saving new weights...\n')
        # 保存模型
        torch.save(model.state_dict(), f'epoch_{t+1}_valid_acc_{(100*valid_acc):0.1f}_model_weights.bin')
print("Done!")

# 它会去保存最好的那个模型

/home/chenli/anaconda3/envs/pytorch/lib/python3.9/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch 1/10
-------------------------------


loss: 0.003257:   0%|          | 2/945 [00:38<5:10:46, 19.77s/it]

RuntimeError: [enforce fail at alloc_cpu.cpp:66] . DefaultCPUAllocator: can't allocate memory: you tried to allocate 1043540064 bytes. Error code 12 (Cannot allocate memory)

# 2022.11.4 训练
用自己的数据集进行训练，不加载模型。<br/>
batch_size=2，共945个批次，轮次=10

In [10]:
from transformers import AdamW, get_scheduler

learning_rate = 1e-5 # 定义学习率
epoch_num = 10 # 轮次定义

loss_fn = nn.CrossEntropyLoss() # 损失函数，交叉熵
optimizer = AdamW(model.parameters(), lr=learning_rate) # Adamw一个常用的优化器
lr_scheduler = get_scheduler(
    "linear",# 使用线性的方式，慢慢往下降
    optimizer=optimizer,
    num_warmup_steps=0,
    num_training_steps=epoch_num*len(train_dataloader),
)

total_loss = 0.
best_acc = 0.
for t in range(epoch_num):
    print(f"Epoch {t+1}/{epoch_num}\n-------------------------------")
    total_loss = train_loop(train_dataloader, model, loss_fn, optimizer, lr_scheduler, t+1, total_loss)
    valid_acc = test_loop(valid_dataloader, model, mode='Valid')
    if valid_acc > best_acc:
        best_acc = valid_acc
        print('saving new weights...\n')
        # 保存模型
        torch.save(model.state_dict(), f'epoch_{t+1}_valid_acc_{(100*valid_acc):0.1f}_model_weights.bin')
print("Done!")

# 它会去保存最好的那个模型

/home/chenli/anaconda3/envs/pytorch/lib/python3.9/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch 1/10
-------------------------------


loss: 0.000000:   0%|          | 0/945 [00:00<?, ?it/s]

RuntimeError: [enforce fail at alloc_cpu.cpp:66] . DefaultCPUAllocator: can't allocate memory: you tried to allocate 1610612736 bytes. Error code 12 (Cannot allocate memory)

# 2022.11.4
模型加载<br/>
batch_size=1进行训练

In [12]:
from transformers import AdamW, get_scheduler

learning_rate = 1e-5 # 定义学习率
epoch_num = 10 # 轮次定义

loss_fn = nn.CrossEntropyLoss() # 损失函数，交叉熵
optimizer = AdamW(model.parameters(), lr=learning_rate) # Adamw一个常用的优化器
lr_scheduler = get_scheduler(
    "linear",# 使用线性的方式，慢慢往下降
    optimizer=optimizer,
    num_warmup_steps=0,
    num_training_steps=epoch_num*len(train_dataloader),
)

total_loss = 0.
best_acc = 0.
for t in range(epoch_num):
    print(f"Epoch {t+1}/{epoch_num}\n-------------------------------")
    total_loss = train_loop(train_dataloader, model, loss_fn, optimizer, lr_scheduler, t+1, total_loss)
    valid_acc = test_loop(valid_dataloader, model, mode='Valid')
    if valid_acc > best_acc:
        best_acc = valid_acc
        print('saving new weights...\n')
        # 保存模型
        torch.save(model.state_dict(), f'epoch_{t+1}_valid_acc_{(100*valid_acc):0.1f}_model_weights.bin')
print("Done!")

# 它会去保存最好的那个模型

Epoch 1/10
-------------------------------


loss: 0.069808:   0%|          | 1/1889 [01:09<36:15:58, 69.15s/it]

RuntimeError: [enforce fail at alloc_cpu.cpp:66] . DefaultCPUAllocator: can't allocate memory: you tried to allocate 751576752 bytes. Error code 12 (Cannot allocate memory)